In [1]:
from sagemaker import get_execution_role
from sagemaker.session import Session

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = 'car-classifier-us-east-2/python_readable_data/stanford_cars_dataset/SGM_data_and_models'

#IAM execution role that gives SageMaker access to resources in your AWS account.
role = get_execution_role()

Define Model

In [2]:
from sagemaker.tensorflow import TensorFlow

car_classifier = TensorFlow(entry_point='CNN_TF_script.py',
                            role=role,
                            train_volume_size = 30,
                            train_max_run = 600, #seconds to run before terminating
                            py_version = 'py3',
                            train_instance_count=2,
                            train_instance_type='ml.m4.xlarge',                           
                            framework_version = '1.12.0',
                            distributions={'parameter_server': {'enabled': True}}
                           )

Train

In [4]:
%%time
import boto3

# use the region-specific sample data bucket
train_data_location = 's3://{}/data/train_head.csv'.format(bucket)
test_data_location = 's3://{}/data/test.csv'.format(bucket)

car_classifier.fit({'train' : train_data_location,
                   'test' : test_data_location
                  })

INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-scriptmode-2019-03-20-22-33-25-091


2019-03-20 22:33:25 Starting - Starting the training job...
2019-03-20 22:33:26 Starting - Launching requested ML instances......
2019-03-20 22:34:27 Starting - Preparing the instances for training...
2019-03-20 22:35:20 Downloading - Downloading input data...
2019-03-20 22:35:49 Training - Training image download completed. Training in progress..
2019-03-20 22:35:52,510 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-03-20 22:35:52,516 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-03-20 22:35:52,726 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-03-20 22:35:52,727 sagemaker_tensorflow_container.training INFO     Launching parameter server process
2019-03-20 22:35:52,727 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-03-20 22:35:52,739 sagemaker_tensorflow_container.training INFO  

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:loss = 0.7010521, step = 0
INFO:tensorflow:loss = 0.7010521
INFO:tensorflow:loss = 0.5675597, step = 4
INFO:tensorflow:loss = 0.5675597
INFO:tensorflow:loss = 0.6997348, step = 11 (1.898 sec)
INFO:tensorflow:loss = 0.6997348 (1.898 sec)
INFO:tensorflow:global_step/sec: 5.85183
INFO:tensorflow:global_step/sec: 5.28442
INFO:tensorflow:loss = 0.5744369, step = 23 (2.521 sec)
INFO:tensorflow:loss = 0.5744369 (2.521 sec)
INFO:tensorflow:global_step/sec: 5.42893
INFO:tensorflow:loss = 0.5122644, step = 35 (2.014 sec)
INFO:tensorflow:loss = 0.5122644 (2.014 sec)
INFO:tensorflow:global_step/sec: 6.52419
INFO:tensorflow:loss = 0.58737844, step = 47 (1.799 sec)
INFO:tensorflow:loss = 0.58737844 (1.800 sec)
INFO:tensorflow:global_step/sec: 6.66308


INFO:tensorflow:global_step/sec: 7.00235
INFO:tensorflow:loss = 0.34978604, step = 526 (9.013 sec)
INFO:tensorflow:loss = 0.34978604 (9.013 sec)
INFO:tensorflow:loss = 0.6131959, step = 530 (1.774 sec)
INFO:tensorflow:loss = 0.6131959 (1.774 sec)
INFO:tensorflow:global_step/sec: 6.82675
INFO:tensorflow:loss = 0.5099561, step = 542 (1.830 sec)
INFO:tensorflow:loss = 0.5099561 (1.831 sec)
INFO:tensorflow:global_step/sec: 5.97082
INFO:tensorflow:global_step/sec: 6.574
INFO:tensorflow:loss = 1.2156281, step = 554 (1.891 sec)
INFO:tensorflow:loss = 1.2156281 (1.891 sec)
INFO:tensorflow:global_step/sec: 6.7972
INFO:tensorflow:loss = 0.56577444, step = 567 (1.899 sec)
INFO:tensorflow:loss = 0.56577444 (1.899 sec)
INFO:tensorflow:global_step/sec: 6.49417
INFO:tensorflow:loss = 0.78866637, step = 579 (1.888 sec)
INFO:tensorflow:loss = 0.78866637 (1.888 sec)
INFO:tensorflow:global_step/sec: 6.43406
INFO:tensorflow:loss = 0.48549828, step = 584 (8.935 sec)
INFO:tensorflow:loss = 0.48549828 (8.935


2019-03-20 22:38:51 Uploading - Uploading generated training modelINFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-20-22:38:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from s3://sagemaker-us-east-2-050922845674/sagemaker-tensorflow-scriptmode-2019-03-20-22-23-43-146/model/model.ckpt-1002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-20-22:38:45
INFO:tensorflow:Saving dict for global step 1002: eval_accuracy = 0.95, global_step = 1002, loss = 0.32601368
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1002: s3://sagemaker-us-east-2-050922845674/sagemaker-tensorflow-scriptmode-2019-03-20-22-23-43-146/model/model.ckpt-1002
INFO:tensorflow:Loss for final step: 1.6814394.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export 

Deploy from object

In [5]:
%%time
car_predictor = car_classifier.deploy(initial_instance_count=1,
                                      instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-scriptmode-2019-03-20-22-33-25-091
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-scriptmode-2019-03-20-22-33-25-091


---------------------------------------------------!CPU times: user 274 ms, sys: 18.6 ms, total: 292 ms
Wall time: 4min 18s


Deploy from file

In [23]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://sagemaker-us-east-2-050922845674/' +
              'sagemaker-tensorflow-scriptmode-2019-03-20-22-33-25-091/output/model.tar.gz', role=role)

predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-serving-2019-03-20-22-59-50-675
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-serving-2019-03-20-22-59-50-675


---------------------------------------------------------------!

Predict

In [12]:
import numpy as np

head_line = np.loadtxt('/home/ec2-user/SageMaker/train_head.csv', delimiter=',')

head_line = head_line[:, :-2].reshape((30, 200, 200, 1))

In [22]:
#object
car_predictor.predict(head_line[0,:])

{'predictions': [{'probabilities': [1.0, 0.0],
   'logits': [120.136, -124.957],
   'classes': 0}]}

In [24]:
#file
car_predictor.predict(head_line[0,:])

{'predictions': [{'probabilities': [1.0, 0.0],
   'logits': [120.136, -124.957],
   'classes': 0}]}

Clean up

In [1]:
import boto3

client = boto3.client('sagemaker')

In [5]:
client.delete_endpoint(EndpointName = 'sagemaker-tensorflow-scriptmode-2019-03-20-22-33-25-091')

{'ResponseMetadata': {'RequestId': 'fd14ade2-dfe2-4589-bf3f-d95f44b3746f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fd14ade2-dfe2-4589-bf3f-d95f44b3746f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 20 Mar 2019 23:09:15 GMT'},
  'RetryAttempts': 0}}

In [6]:
client.delete_model(ModelName = 'sagemaker-tensorflow-serving-2019-03-20-22-59-50-675')

{'ResponseMetadata': {'RequestId': '0de5a353-b9ec-446a-9039-0a813fbda7bb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0de5a353-b9ec-446a-9039-0a813fbda7bb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 20 Mar 2019 23:09:33 GMT'},
  'RetryAttempts': 0}}